# Deep Co-segmentation Experiments

## Imports

In [1]:
from dataset import iCosegDataset, PASCALVOCCosegDataset, MSRCDataset, InternetDataset
from model import SiameseSegNet
import numpy as np
import os
import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm_notebook

## Constants

In [2]:
## Debug

DEBUG = False


## Dataset
BATCH_SIZE = 2 * 1 # two images at a time for Siamese net
INPUT_CHANNELS = 3 # RGB
OUTPUT_CHANNELS = 2 # BG + FG channel

## Inference
CUDA = "0"

## Output Dir
OUTPUT_DIR = "./experiments"

os.system(f"rm -r {OUTPUT_DIR}")
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Metrics

In [3]:
def metrics(pmapA, pmapB, masksA, masksB):
    intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = 0, 0, 0, 0, 0, 0
    
    for idx in range(BATCH_SIZE//2):
        pred_maskA = torch.argmax(pmapA[idx], dim=0).cpu().numpy()
        pred_maskB = torch.argmax(pmapB[idx], dim=0).cpu().numpy()

        masksA_cpu = masksA[idx].cpu().numpy()
        masksB_cpu = masksB[idx].cpu().numpy()

        intersection_a += np.sum(pred_maskA & masksA_cpu)
        intersection_b += np.sum(pred_maskB & masksB_cpu)

        union_a += np.sum(pred_maskA | masksA_cpu)
        union_b += np.sum(pred_maskB | masksB_cpu)

        precision_a += np.sum(pred_maskA == masksA_cpu)
        precision_b += np.sum(pred_maskB == masksB_cpu)

    return intersection_a, intersection_b, union_a, union_b, precision_a, precision_b

# Experiments

## Load Deep Object Co-segmentation model trained on Pascal VOC 

In [4]:
LOAD_CHECKPOINT = "/home/SharedData/intern_sayan/PASCAL_coseg/"

model = SiameseSegNet(input_channels=INPUT_CHANNELS,
                          output_channels=OUTPUT_CHANNELS,
                          gpu=CUDA)
if DEBUG:
    print(model)

FloatTensor = torch.FloatTensor
LongTensor = torch.LongTensor

if CUDA is not None:
    os.environ["CUDA_VISIBLE_DEVICES"] = CUDA

    model = model.cuda()

    FloatTensor = torch.cuda.FloatTensor
    LongTensor = torch.cuda.LongTensor

if LOAD_CHECKPOINT:
    model.load_state_dict(torch.load(os.path.join(LOAD_CHECKPOINT, "coseg_model_best.pth")))

## iCoseg

In [5]:
root_dir = "/home/SharedData/intern_sayan/iCoseg/"

image_dir = os.path.join(root_dir, "images")
mask_dir = os.path.join(root_dir, "ground_truth")

dataset = iCosegDataset(image_dir=image_dir,
                        mask_dir=mask_dir)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)

## VOC + iCoseg [Car]  

iCoseg class indices = {5}

In [6]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([5,5]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"car_iCoseg_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"car_iCoseg_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"car_iCoseg_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [4.99291729927063 secs]
Precision : [0.01391393596138167]
IoU : [0.5743049860244029]


## VOC + iCoseg [People]
iCoseg class indices = {1,4,26,27,28}

In [7]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([1,1]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([4,4]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([26,26]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([26,27]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([27,27]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([27,28]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([28,28]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"people_iCoseg_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"people_iCoseg_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"people_iCoseg_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [25.31199550628662 secs]
Precision : [0.1715789093778141]
IoU : [0.41228012935962516]


## VOC + iCoseg [Goose]
iCoseg class indices = {10}

In [8]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([10,10]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"goose_iCoseg_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"goose_iCoseg_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"goose_iCoseg_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [8.782491445541382 secs]
Precision : [0.03810905147564374]
IoU : [0.38239511332035714]


## VOC + iCoseg [Airplane]  

iCoseg class indices = {12,13,14}

In [9]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([12,12]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([12,13]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([13,13]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([13,14]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([14,14]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"airplane_iCoseg_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"airplane_iCoseg_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"airplane_iCoseg_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [15.402763605117798 secs]
Precision : [0.108995289074669]
IoU : [0.3660986589923934]


### Clean-up

In [10]:
del dataset
del dataloader

## MSRC Dataloader

In [11]:
root_dir = "/home/SharedData/intern_sayan/MSRC_processed/"

image_dir = os.path.join(root_dir, "images")
mask_dir = os.path.join(root_dir, "GT")

dataset = MSRCDataset(image_dir=image_dir,
                      mask_dir=mask_dir)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)

## VOC + MSRC [Car]  

MSRC class indices = {1}

In [12]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([1,1]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"car_MSRC_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"car_MSRC_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"car_MSRC_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [6.370521306991577 secs]
Precision : [0.0685278240003084]
IoU : [0.3403835207992279]


## VOC + MSRC [Airplane]  

MSRC class indices = {5}

In [13]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([5,5]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"airplane_MSRC_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"airplane_MSRC_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"airplane_MSRC_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [7.229506015777588 secs]
Precision : [0.061545257933402174]
IoU : [0.47602728675268097]


## VOC + MSRC [Bird]  

MSRC class indices = {7}

In [14]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([7,7]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"bird_MSRC_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"bird_MSRC_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"bird_MSRC_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [8.12025260925293 secs]
Precision : [0.05666641527385803]
IoU : [0.43510671340462354]


## VOC + MSRC [Cat]  

MSRC class indices = {8}

In [15]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([8,8]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"bird_MSRC_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"bird_MSRC_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"bird_MSRC_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [6.225841522216797 secs]
Precision : [0.05170665175150456]
IoU : [0.18139940688218698]


## VOC + MSRC [Cow]  

MSRC class indices = {9}

In [16]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([9,9]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"bird_MSRC_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"bird_MSRC_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"bird_MSRC_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [5.706972599029541 secs]
Precision : [0.0424988486550071]
IoU : [0.009847018975376588]


### Clean-up

In [17]:
del dataset
del dataloader

## Internet Dataloader

In [18]:
root_dir = "/home/SharedData/intern_sayan/internet_processed/"

image_dir = os.path.join(root_dir, "images", "Data")
mask_dir = os.path.join(root_dir, "GT", "Data")

dataset = InternetDataset(image_dir=image_dir,
                          mask_dir=mask_dir)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)

## VOC + Internet [Airplane]  

MSRC class indices = {0}

In [19]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([0,0]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"airplane_Internet_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"airplane_Internet_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"airplane_Internet_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [14.892923831939697 secs]
Precision : [0.45864110946655273]
IoU : [0.42364249199214404]


## VOC + MSRC [Car]  

MSRC class indices = {1}

In [20]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([1,1]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"car_Internet_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"car_Internet_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"car_Internet_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [16.340217113494873 secs]
Precision : [0.4201101875305176]
IoU : [0.5177999570587426]


### Clean-up

In [21]:
del dataset
del dataloader